In [1]:
from fewshot.data_provider.dataset import Dataset
from fewshot.backbones import ConvNet

from fewshot.algorithms.backbone_pretrain import simple_one_layer_cross_entropy_train
from fewshot.algorithms.fewshot_models import BaselineFewShotModel
from tensorflow.keras import callbacks

import numpy as np
import tqdm
import scipy.stats as st

import tensorflow as tf

from IPython.display import clear_output

### Init

Set up all input data and params

In [2]:
# common
seed = 11

img_width = 84
img_height = 84
img_depth = 3

# classes
base_num_classes = 64
val_num_classes = 16  # not used in baseline algo
novel_num_classes = 20

# backbone training
backbone_training_batch_size = 32
backbone_training_epochs = 2

# fewshot training and testing
n_way = 5
k_shot = 5
training_batch_size = 4
training_batches_per_episode = 25
query_samples_per_class = 16

n_episodes = 50

In [3]:
dataset_dir = "../../../data/mini_imagenet/"
dataset = Dataset(dataset_dir=dataset_dir, csv_name="data_84x84.csv", image_size=(img_width, img_height))

### Backbone
Create backbone dataset (80 classes), split to train and validattion

In [4]:
backbone_dataset, val_fewshot_dataset = dataset.split_by_classes(train_size=base_num_classes,
                                                                 random_state=seed)

Split by classes with train size = 64 (seed = 11)
Train classes: 64
Test classes: 36
Train data: 38400 samples
Test data:  21600 samples


/mnt/disk/venv/dpy3/lib/python3.5/site-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [5]:
val_dataset, fewshot_dataset = val_fewshot_dataset.split_by_classes(train_size=val_num_classes,
                                                                    random_state=seed)

Split by classes with train size = 16 (seed = 11)
Train classes: 16
Test classes: 20
Train data: 9600 samples
Test data:  12000 samples


Create classifier

In [6]:
backbone = ConvNet(input_size=(img_width, img_height, img_depth))

Instructions for updating:
Colocations handled automatically by placer.


Load weights if we have presaved

Train

In [7]:
backbone.set_trainable(True)

In [21]:
backbone_callbacks = [
    tf.keras.callbacks.ModelCheckpoint("../../../data/few-models/weights.{epoch:02d}-{loss:.2f}.hdf5",
                                       monitor="loss",
                                       save_best_only=False)
]

In [ ]:
simple_one_layer_cross_entropy_train(
    backbone,
    backbone_dataset.get_batch_generator(batch_size=backbone_training_batch_size, shuffle=True),
    n_epochs=backbone_training_epochs,
    callbacks=backbone_callbacks
)

### Few Shot
Train fewshot model

In [ ]:
accs = []
for episode in tqdm.tqdm_notebook(range(n_episodes)):
    fewshot_model = BaselineFewShotModel(backbone, n_way)
    
    # not very simple way to prepare eposide support and query sets
    # FIXME: in future replace it with @bobbythehiver .few_shot_episode_generator() implementation
    # and @schoooler tester version
    episode_dataset, _ = fewshot_dataset.split_by_classes(train_size=n_way, random_state=episode)
    
    # subset support and query datasets
    episode_support, episode_left = episode_dataset.split_by_objects(train_size=k_shot, random_state=episode)
    episode_query, _ = episode_left.split_by_objects(train_size=query_samples_per_class, random_state=episode)
    
    
    fewshot_model.fit(episode_support.get_batch_generator(batch_size=training_batch_size, shuffle=True),
                      n_epochs=training_batches_per_episode)
    
    # calc accuracy
    out = fewshot_model.predict(episode_query.get_batch_generator(batch_size=training_batch_size, shuffle=False))
    classes = np.array(episode_query.classes)
    acc = np.mean(classes[np.argmax(out, axis=1)] == episode_query.dataframe["class"].values)
    
    accs.append(acc)
    clear_output()
    
    print("Mean accuracy (by {} episodes): {:.2f}%".format(episode + 1, np.mean(accs) * 100))

### Report

Calc 95% confidence interval and report

In [ ]:
acc_int = st.t.interval(0.95, len(accs) - 1, loc=np.mean(accs), scale=st.sem(accs))

In [ ]:
print("5-shot test metric: {}% +- {}%".format(round(np.mean(accs) * 100, 2),
                                              round((np.mean(acc_int) - acc_int[0]) * 100, 2)))